<a href="https://colab.research.google.com/github/lise1905/mini_project/blob/main/clothing_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install surprise

     |████████████████████████████████| 11.8 MB 5.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623246 sha256=5d4480948b0f7306f62802ed17967d98457757f5329b35a5056b21f6ba51a940
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np
import os, io
from surprise import SVD
from surprise import accuracy
import csv

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data = pd.read_json('renttherunway_final_data.json', lines=True)
train, test = train_test_split(data, test_size=0.2)

In [10]:
print(train.head(10))

          fit  user_id bust size  item_id  ... height  size   age        review_date
62628     fit   148775       34b   727157  ...  6' 0"     8  33.0   January 19, 2017
130456    fit   670592       34b  2318793  ...  5' 3"     8  31.0    October 3, 2017
161758  small   451231       34c  1973331  ...  5' 6"    14  31.0    August 21, 2016
63399   large   483792       36a   174086  ...  5' 5"    16  37.0      July 14, 2015
42028     fit   891326       32d   188485  ...  5' 2"     8  33.0   January 21, 2016
101596  large   362951   34ddd/e  2282065  ...  5' 5"    20  23.0      July 19, 2017
177366  small   306184       34b  1158451  ...  5' 4"     8  34.0       May 18, 2016
8589    large   178484       36b   710897  ...  5' 7"    24  31.0    August 18, 2017
140168    fit   417878       34c  1333316  ...  5' 6"    14  31.0   October 19, 2016
11992     fit   500780       36d   427952  ...  5' 7"    28  39.0  September 6, 2017

[10 rows x 15 columns]


In [11]:
print(test.head(10))

          fit  user_id bust size  ...  size   age         review_date
33477     fit   783293       32d  ...     4  32.0       June 25, 2015
106975    fit   852706       32d  ...     5  34.0     January 9, 2017
7036      fit   285836       32a  ...     1  24.0    January 10, 2017
27334     fit   448198       32c  ...     8  50.0       March 7, 2017
125994    fit   267250       34b  ...     8  25.0      April 17, 2017
122422    fit   111436       32a  ...     1  32.0     August 31, 2016
67666   large   275028      36d+  ...    24  32.0  September 21, 2014
138876    fit   431770       34b  ...     1  42.0     October 5, 2014
83467     fit   958548       36a  ...    14  35.0       June 13, 2017
123337    fit   359031       34b  ...    12  26.0         May 1, 2015

[10 rows x 15 columns]


In [4]:
# Exception Handeling for Data Sorting in pandas
pd.options.mode.chained_assignment = None

In [5]:
def preprocessAge(data):
  bins = pd.IntervalIndex.from_tuples([(5, 15), (15, 30), (30, 50),(50,65),(65,100)])
  x = pd.cut(data['age'].to_list(), bins)
  x.categories = ['1','2','3','4','5']
  data['age_bins']  = x
  age_Preprocessed_data = data[~data['age_bins'].isnull()]
  # print ('{0} rows got dropped after age preprocessing'.format(str(data.shape[0] - age_Preprocessed_data.shape[0])))
  return age_Preprocessed_data

In [6]:
def preprocessBustsize(age_Preprocessed_data):
  data_bust_size_not_null = age_Preprocessed_data[~age_Preprocessed_data['bust size'].isnull()]
  data_bust_size_not_null['bust size'] = data_bust_size_not_null['bust size'].apply(lambda x : (int(x[:2]), x[2:]))
  data_bust_size_not_null.shape
  data_bust_size_not_null[['bust Size1', 'bust']] = pd.DataFrame(data_bust_size_not_null['bust size'].tolist(), index=data_bust_size_not_null.index)
  bustDict = {'a': 1, 'aa': 1, 'b': 2, 'c': 3, 'd': 4, 
            'd+': 5, 'dd': 5, 'ddd/e': 6,
           'f':7 , 'g': 8, 'h': 9, 'i': 10, 'j': 11}
  data_bust_size_not_null['bust1'] = data_bust_size_not_null['bust'].map(bustDict)
  # Calculating model of bust size based on age grp.
  def funct(df):
    return df.mode()
  x = data_bust_size_not_null[['age_bins','bust Size1','bust1']].groupby('age_bins').apply(funct)
  x.reset_index(inplace= True, drop = True)
  data_bust_size_null = age_Preprocessed_data[age_Preprocessed_data['bust size'].isnull()]
  data_bust_size_null = pd.merge(data_bust_size_null, x, how ='left', left_on = 'age_bins', right_on = 'age_bins')
  data_bust_size_cleansed = data_bust_size_not_null.append(data_bust_size_null)
  return data_bust_size_cleansed

In [7]:
def preprocessHeight(data_bust_size_cleansed):
  data_bust_size_cleansed['heightCM'] = data_bust_size_cleansed['height'].apply(lambda x : (int(x.split("\'")[0]) * 30.48) + (int(x.split("\'")[1][:-1]) * 2.54) 
                                                        if type(x) == str else x)
  data_bust_size_cleansed['heightCM'].fillna((data_bust_size_cleansed['heightCM'].mean()), inplace=True)
  return data_bust_size_cleansed

In [8]:
def preprocessWeight(data_bust_size_cleansed):
  data_bust_size_cleansed['weightLbs'] = data_bust_size_cleansed['weight'].apply(lambda x : int(x[:-3])                                                        if type(x) == str else x)
  data_bust_size_cleansed['weightLbs'].fillna((data_bust_size_cleansed['weightLbs'].mean()), inplace=True)
  return data_bust_size_cleansed